In [3]:
import numpy as np
from pymoo.core.problem import ElementwiseProblem
import optimize as op

class MyProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=2,
                         n_obj=2,
                         n_ieq_constr=2,
                         xl=np.array([-2,-2]),
                         xu=np.array([2,2]))

    def _evaluate(self, x, out, *args, **kwargs):
        f1 = 100 * (x[0]**2 + x[1]**2)
        f2 = (x[0]-1)**2 + x[1]**2

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]


problem = MyProblem()

opt = op.corgiOptimize()
opt.getLiftoffState()
print(opt.velocityConstraint(opt.bp_init_guess))
print(opt.multiObjective(opt.bp_init_guess))


class CorgiProblem(ElementwiseProblem):

    def __init__(self):
        super().__init__(n_var=24,
                         n_obj=2,
                         n_ieq_constr=5,
                         xl=np.array([0.01, 0.00, 0.01, -0.1, 0.01, -0.1]),
                         xu=np.array([0.03, 0.02, 1, 0.1, 1, 0.1]))

    def _evaluate(self, x, out, *args, **kwargs):
        cost_s, cost_u = opt.multiObjective(x)
        f1 = cost_s
        f2 = cost_u

        g1 = 2*(x[0]-0.1) * (x[0]-0.9) / 0.18
        g2 = - 20*(x[0]-0.4) * (x[0]-0.6) / 4.8
        # g3 =
        # g4 =
        # g5 =

        out["F"] = [f1, f2]
        out["G"] = [g1, g2]


problem = MyProblem()

[-0.84949204  0.07409333 -0.83966248 -0.11109228 -0.84949204  0.07409333
 -0.83966248 -0.11109228 -0.84949204  0.07409333 -0.83966248 -0.11109228
 -0.84949204  0.07409333 -0.83966248 -0.11109228]
iter 1 cost -4.841397181844985
[-0.8671381233165221, 0.4206900320402545]


In [4]:
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling

algorithm = NSGA2(
    pop_size=40,
    n_offsprings=10,
    sampling=FloatRandomSampling(),
    crossover=SBX(prob=0.9, eta=15),
    mutation=PM(eta=20),
    eliminate_duplicates=True
)

In [5]:
from pymoo.termination import get_termination

termination = get_termination("n_gen", 40)

In [6]:
from pymoo.optimize import minimize

res = minimize(problem,
               algorithm,
               termination,
               seed=1,
               save_history=True,
               verbose=True)

X = res.X
F = res.F

n_gen  |  n_eval  | n_nds  |     cv_min    |     cv_avg    |      eps      |   indicator  
     1 |       40 |      1 |  0.000000E+00 |  2.363992E+01 |             - |             -
     2 |       50 |      1 |  0.000000E+00 |  1.262978E+01 |  0.000000E+00 |             f
     3 |       60 |      2 |  0.000000E+00 |  5.3749132447 |  1.0000000000 |         ideal
     4 |       70 |      2 |  0.000000E+00 |  2.4309071542 |  0.000000E+00 |             f
     5 |       80 |      3 |  0.000000E+00 |  0.9326371456 |  0.0096178563 |         ideal
     6 |       90 |      3 |  0.000000E+00 |  0.1066499133 |  0.0601661450 |         ideal
     7 |      100 |      3 |  0.000000E+00 |  0.0092174421 |  0.000000E+00 |             f
     8 |      110 |      5 |  0.000000E+00 |  0.0010944863 |  0.2816759453 |         ideal
     9 |      120 |      6 |  0.000000E+00 |  0.000000E+00 |  0.0124399889 |             f
    10 |      130 |      9 |  0.000000E+00 |  0.000000E+00 |  0.0041045275 |         ideal

In [7]:
print(X)

[[ 0.1596701  -0.01898445]
 [ 0.89312273 -0.00097933]
 [ 0.61843457 -0.01406674]
 [ 0.3970102  -0.0715932 ]
 [ 0.19736088 -0.01898445]
 [ 0.27550335 -0.00108155]
 [ 0.76183795  0.0242858 ]
 [ 0.22846855 -0.01945366]
 [ 0.79625866  0.00293492]
 [ 0.172439    0.04500609]
 [ 0.30448793 -0.04784723]
 [ 0.3387105   0.03135848]
 [ 0.70993162 -0.04061043]
 [ 0.8623285  -0.07581915]
 [ 0.88062313 -0.03906068]
 [ 0.25338728 -0.01568633]
 [ 0.31888571 -0.06625216]
 [ 0.69246504 -0.01198123]
 [ 0.82924476 -0.01406674]
 [ 0.37286488  0.01794183]
 [ 0.63074713 -0.04640053]
 [ 0.38700971 -0.01198123]
 [ 0.64345227  0.03098593]
 [ 0.817646   -0.0436996 ]
 [ 0.35346693 -0.07679382]
 [ 0.72666401 -0.03013668]
 [ 0.6821802  -0.01281022]
 [ 0.65517858  0.05955645]
 [ 0.8046009   0.008906  ]
 [ 0.36181388 -0.00782364]
 [ 0.75171907  0.03825816]
 [ 0.2381627   0.01959426]
 [ 0.74227363  0.06354789]
 [ 0.67044566 -0.01281022]
 [ 0.84653089 -0.07029015]
 [ 0.84498794 -0.07042649]
 [ 0.24663591 -0.01898445]
 